In [39]:
import pandas as pd
import requests
from tqdm import tqdm

In [40]:
#load data
errorResearchers = pd.read_pickle("Files/DataFrames/researcherNotFound.pkl")

In [41]:
#errorResearchers names without departmnet/university/group/team
errorResearchers_ = errorResearchers[~errorResearchers.SearchName.str.contains('department|university|group|team|collaboration')]

In [97]:
#drop first index, which we can see is an error
errorResearchers_df = errorResearchers_.drop(0).reset_index(drop = True)

In [98]:
#save researchers as researchers_allFrom2005
errorResearchers_df.to_pickle("Files/Dataframes/errorResearchers_filtered.pkl")

### Estimate gender

In [99]:
def assignGender(df, NameCol): #from GenderEstimation_theis.ipynb
    index = 0
    nameDict = {}
    genderEstimation = pd.read_csv("Files/wgnd_2_0_name-gender-code.csv") #genderEstimation file
    genderEstimation['Name'] = genderEstimation.name.str.replace(r'"|\'', '', regex=True).str.lower() #remove " and ' and make to lower case
    
    #firstName = getFirstName(df, 'Name') #find first names
    #df['FirstName'] = firstName #add first name to df
    #df[NameCol] = df.FirstName.str.lower() #first name, but lower case
  
    usNames = genderEstimation[genderEstimation.code == 'US'] #some names can't not be found, but can with another code
    otherNames = genderEstimation[~genderEstimation.Name.isin(usNames.Name)] #names not in US

    uniqueNamesUS = df[df[NameCol].isin(usNames.Name)][NameCol].unique() #df with names from presenters, that is also in genderEstimation
    uniqueNamesOther = df[df[NameCol].isin(otherNames.Name)][NameCol].unique()
    
    for n in tqdm(uniqueNamesUS): #loop through us names that is also in the presenter df
        nameTable = usNames[usNames.Name == n] #table with the name 
        genderIndex = nameTable.wgt.argmax() #index of gender with highest score/probability
        gender = nameTable.iloc[genderIndex].gender #gender that is most likely
        nameDict[index] = {'firstName': n, 'Gender': gender} #assign gender to the name in a dict
        index += 1
    
    for m in tqdm(uniqueNamesOther): #loop through other names that is also in the presenter df
        nameTable = otherNames[otherNames.Name == m].groupby('gender').mean(numeric_only = True) #table with the name - mean of wgt
        genderIndex = nameTable.wgt.argmax() #index of gender with higest average probability
        gender = nameTable.iloc[genderIndex].name
        nameDict[index] = {'firstName': m, 'Gender': gender} #assign gender to the name in a dict
        index += 1
    
    return nameDict

In [100]:
def getFirstName(name):
    firstName = name.split()[0]
    
    return firstName

In [101]:
errorResearchers_df['FirstName'] = errorResearchers_df.apply(lambda x: getFirstName(x.SearchName), axis = 1)

In [102]:
genderDict = assignGender(errorResearchers_df,'FirstName')

100%|███████████████████████████████████████| 3047/3047 [08:01<00:00,  6.33it/s]


In [103]:
#dataframe witht the gender and name
genderAssign = pd.DataFrame.from_dict(genderDict, orient = 'index')

Now we are merging genderAssign and errorResearchers_df. This will automatically add the gender to names to the researchers. 

In [133]:
#merge error researcher df with gender df
errorResearchers_wGender_df = errorResearchers_df.merge(genderAssign, how = 'left', left_on = 'FirstName', right_on = 'firstName').drop('firstName', axis = 1)

In [145]:
print("Number of unique names not assigned to gender", len(errorResearchers_wGender_df[errorResearchers_wGender_df.Gender.isna()].SearchName.unique()))
#print("Numberlen(errorResearchers_wGender_df[errorResearchers_wGender_df.Gender.isna()])

Number of unique names not assigned to gender 5922


In [146]:
#remove names with gender assigned to NaN
errorResearchers_wGender = errorResearchers_wGender_df[~errorResearchers_wGender_df.Gender.isna()]

In [147]:
errorResearchers_wGender.to_pickle("Files/Dataframes/ResearchersNotFound_wGender.pkl") 

In [164]:
print("Number of unique names assigned to ? as gender:", len(errorResearchers_wGender[errorResearchers_wGender.Gender == '?'].SearchName.unique()))
print("Number of names assigned to gender", len(errorResearchers_wGender[(errorResearchers_wGender.Gender == 'F') | (errorResearchers_wGender.Gender == 'M')]))
print("Number of unique names assigned to gender", len(errorResearchers_wGender[(errorResearchers_wGender.Gender == 'F') | (errorResearchers_wGender.Gender == 'M')].SearchName.unique()))

Number of unique names assigned to ? as gender: 605
Number of names assigned to gender 117383
Number of unique names assigned to gender 35206


In [111]:
errorResearchers_wGender
#.iloc[27]

,ID,SearchName,Year,Name,Institution,AuthorIndex,Title,Division,SessionIndex,SessionType,LowerCaseName,FirstName,Gender
0,100007,swathy natarajan,2023,Swathy Natarajan,"University of California, Berkeley",5.0,Effects of dimensionality on the magnetic beha...,GMAG,64672.0,attendee,swathy natarajan,swathy,F
1,100009,thomas j blommel,2022,Thomas J Blommel,University of Michigan,1.0,Equilibrium Spectral Functions from Finite-tem...,DCOMP DAMOP DCMP,64674.0,attendee,thomas j blommel,thomas,M
2,100009,thomas j blommel,2023,Thomas J Blommel,University of Michigan,3.0,Quantum Monte Carlo for multi-orbital systems ...,DCOMP DAMOP DCMP,109382.0,attendee,thomas j blommel,thomas,M
3,100042,carolin roskothen,2021,Carolin Roskothen,Goethe University Frankfurt,1.0,Envy in competitive societies induces a class-...,GSNP,64710.0,attendee,carolin roskothen,carolin,F
4,100068,ryan tiew,2023,Ryan Tiew,University of Bristol,3.0,Circuit QED signatures of Majorana bound state...,DMP,64730.0,attendee,ryan tiew,ryan,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134791,99973,goran krilov,2021,Goran Krilov,Schrodinger Inc,1.0,An efficient approach to computing drug-target...,DCP,64650.0,attendee,goran krilov,goran,M
134792,9998,xianxu zheng,2020,Xianxu Zheng,National Key Laboratory of Shock Wave and Deto...,8.0,Dance of HMX molecule in conformational space ...,GSCCM,9104.0,attendee,xianxu zheng,xianxu,NaN
134793,99993,stephanie anderson,2017,Stephanie Anderson,Dow Chemical Co,6.0,Impact of x-Linkable Polymer Blends on Phase M...,DPOLY,64658.0,attendee,stephanie anderson,stephanie,F
134794,99994,colin li pi shan,2017,Colin Li Pi Shan,Dow Chemical Co,7.0,Impact of x-Linkable Polymer Blends on Phase M...,DPOLY,64658.0,attendee,colin li pi shan,colin,M


In [116]:
#errorResearchers_wGender[errorResearchers_wGender.Gender.isna()]
#Female: 21411
#Male: 95972 
#?: 1827
errorResearchers_wGender[errorResearchers_wGender.Gender.isna()]

#delete: 26, 27, 30, 41, 42, 134780, 

#check every place where Institution is Unknown, it could look like the institution is saved as name. 


#check how big a part of the unknown, that are invited, and how the gender distributes here

,ID,SearchName,Year,Name,Institution,AuthorIndex,Title,Division,SessionIndex,SessionType,LowerCaseName,FirstName,Gender
26,10025,sensor cat at stony brook,2016,Sensor CAT at Stony Brook,Unknown,5.0,High-Energy-Density Cost-Effective Graphene Su...,GERA FIAP,9113.0,attendee,sensor cat at stony brook,sensor,NaN
27,10025,sensor cat at stony brook,2015,Sensor CAT at Stony Brook,Unknown,4.0,High Performance and Economic Supercapacitors ...,GERA DMP FIAP,152374.0,attendee,sensor cat at stony brook,sensor,NaN
30,100301,qnstl,2022,QNSTL,Unknown,6.0,Cotunneling assisted internal relaxation trans...,DCMP,64949.0,attendee,qnstl,qnstl,NaN
41,100421,"gmag, hugtp, upc",2013,"GMAG, HUGTP, UPC",Unknown,8.0,Microwave Spectrometry for the Assessment of t...,GIMS,65055.0,attendee,"gmag, hugtp, upc","gmag,",NaN
42,100443,organic thin films,2007,Organic Thin Films,Unknown,3.0,Molecular Scale Structure of Pentacene Interfaces,DPOLY DMP,65078.0,attendee,organic thin films,organic,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134751,99840,tharangattu n. narayanan,2023,Tharangattu N. Narayanan,"Centre for Interdisciplinary Sciences, Tata In...",5.0,Novel Experimental Method to Measure the Stati...,DSOFT GSNP,64525.0,attendee,tharangattu n. narayanan,tharangattu,NaN
134757,99849,kyoung-keun lee,2005,Kyoung-Keun Lee,Georgia Institute of Technology,1.0,III-Nitrides on Ferroelectric Lithium Niobate:...,DMP,64534.0,attendee,kyoung-keun lee,kyoung-keun,NaN
134780,99948,theoretical chemistry,2015,Theoretical Chemistry,Unknown,3.0,Latent heat of magnetization for MnFeSi0.33P0.66,DCMP,64631.0,attendee,theoretical chemistry,theoretical,NaN
134781,9995,rucheng dai,2020,Rucheng Dai,"The Centre for Physical Experiments, Universit...",5.0,Dance of HMX molecule in conformational space ...,GSCCM,9104.0,attendee,rucheng dai,rucheng,NaN


In [180]:
parameters = {
            "query": "Jeremie Guillaud",
            "fields": "name,paperCount,citationCount,hIndex,aliases,papers.title,papers.fieldsOfStudy,papers.s2FieldsOfStudy,papers.referenceCount,papers.year,papers.citationCount,papers.influentialCitationCount,papers.publicationDate"
            }
url = "https://api.semanticscholar.org/graph/v1/author/search?&limit=1"
APIkey = {'x-api-key': 'tRss0MQnxJaaE0KmrqBUr7gGK2Y7Bm8C8UdCCLzP'}

response = requests.get(url, params = parameters, headers = APIkey, timeout = 5).json()

In [181]:
response

{'total': 0, 'offset': 0, 'data': []}

In [179]:
errorResearchers[errorResearchers.SessionType == 'invited']

,ID,SearchName,Year,Name,Institution,AuthorIndex,Title,Division,SessionIndex,SessionType,LowerCaseName
7,100069,youtarou takahash,2022,Youtarou Takahash,University of Tokyo,1.0,Nonreciprocal optical responses of multiferroics,DMP,64731.0,invited,youtarou takahash
80,100611,"olivia fermi, m. a.",2014,"Olivia Fermi, M. A.",None,1.0,Then and Now: Women Respond to the Manhattan P...,None,65232.0,invited,"olivia fermi, m. a."
396,103318,jeremie guillaud,2020,Jeremie Guillaud,"Quantic, Inria Paris",1.0,Fault-tolerant quantum computation with repeti...,DQI,138317.0,invited,jeremie guillaud
397,103318,jeremie guillaud,2021,Jeremie Guillaud,Alice&Bob,1.0,Hardware efficient quantum computing using rep...,DQI,67741.0,invited,jeremie guillaud
481,104499,becker sharif,2023,Becker Sharif,"University of California, Santa Cruz",1.0,TBD,FGSA,110138.0,invited,becker sharif
...,...,...,...,...,...,...,...,...,...,...,...
137080,99869,rutger hermsen,2011,Rutger Hermsen,"Center for Theoretical Biological Physics, Uni...",1.0,Adaptation Driven by Spatial Heterogeneities,DBP DCP,64637.0,invited,rutger hermsen
137094,99914,m. r. stoneking,2014,M. R. Stoneking,"Department of Physics, Lawrence University, Ap...",1.0,Capitalizing on Community: the Small College E...,FEd,64588.0,invited,m. r. stoneking
137105,99967,robin williams,2007,Robin Williams,"Institute for Microstructural Sciences, Nation...",1.0,Directed Self-Assembly of Single Quantum Dot O...,DMP,64645.0,invited,robin williams
137108,99972,juan p. garrahan,2015,Juan P. Garrahan,"School of Physics \& Astronomy, University of ...",1.0,The structure of glass as revealed by dynamica...,DPOLY,103637.0,invited,juan p. garrahan


In [158]:
#Search names containing group 
#errorResearchers[errorResearchers.SearchName.str.contains('group')].SearchName.unique()
errorResearchers[errorResearchers.SearchName.str.contains('department|university|group|team')]

,ID,SearchName,Year,Name,Institution,AuthorIndex,Title,Division,SessionIndex,SessionType,LowerCaseName
4,100018,"purdue univeristy, yong p. chen group",2013,"Purdue Univeristy, Yong P. Chen Group",Unknown,4.0,Effect of gate-induced doping on the Raman spe...,DMP,64683.0,attendee,"purdue univeristy, yong p. chen group"
10,100088,searson group,2013,Searson group,Unknown,5.0,Influence of curvature on the morphology of br...,APS,64751.0,attendee,searson group
11,100113,"dpmc, university of geneva, 24,, department of...",2008,"DPMC, University of Geneva, 24,, Department of...",Unknown,2.0,Optical spectra of the heavy fermion uniaxial ...,DCMP,64775.0,attendee,"dpmc, university of geneva, 24,, department of..."
13,100131,theoretical condensed matter group at ut,2005,Theoretical Condensed Matter Group at UT,Unknown,2.0,Charge-Hall effect driven by spin force: recip...,DMP GMAG,64791.0,attendee,theoretical condensed matter group at ut
14,100146,department of physics - ufrn,2014,Department of Physics - UFRN,Unknown,3.0,Anomalous Nernst Effect with Magnetocrystallin...,None,64813.0,attendee,department of physics - ufrn
...,...,...,...,...,...,...,...,...,...,...,...
134111,94967,university of surrey,2014,University of Surrey,Unknown,3.0,Switchable adhesion of liquid crystalline elas...,DPOLY DBIO GSNP,60059.0,attendee,university of surrey
134790,96292,king saud university,2022,King Saud University,Unknown,3.0,The Phenomenon Cancer,GMED DBIO,61292.0,attendee,king saud university
135612,97097,department of chemical \& biological engineeri...,2009,Department of Chemical \& Biological Engineeri...,Unknown,2.0,New type of Reverse Osmosis Membrane via Layer...,None,61984.0,attendee,department of chemical \& biological engineeri...
136567,98938,university of isfahan,2005,University of Isfahan,Unknown,2.0,The effect of pressure on the electric field g...,None,63709.0,attendee,university of isfahan


In [101]:
#realNames: errorResearchers_df[errorResearchers_df.SearchName.str.contains('/')].iloc[x]
#x = 5, 14, 40, 44
#maybe plus 1


#change searchName to everything after ":", and then explode the cell. 

'[fuzzyics/(spd/m)], additional collaborators: harry suhl, leo brewer, walter munk, norman bowen, frank schairer, andrew carniege, walter kohn, victor goldschmidt, adolph smith, ace hoffman, william smithson, richard wunderma'

In [148]:
errorResearchers_df[errorResearchers_df.SearchName.str.contains('/')].iloc[45]
#ask about what to do


ID                                                          162478
SearchName       edward carl-ludwig (physical-mathematicist/mat...
Year                                                          2011
Name             Edward Carl-Ludwig (Physical-Mathematicist/Mat...
Institution                      Physical-Mathematicist/Mathsicist
AuthorIndex                                                    4.0
Title            Both Perelmans Thrown Down Gauntlets Versus Wo...
Division                                                       APS
SessionIndex                                              137337.0
SessionType                                               attendee
LowerCaseName    edward carl-ludwig (physical-mathematicist/mat...
Name: 6734, dtype: object

In [154]:
#we need to find out what to do with these people. 
#How many do we have left, that we couldnt find?
#Should we just use API, or should we try some cleaning?
#Can we use explode in some way and then see how we can find?

'marvin antonoff/albert overhauser(rip)/frederic young/adolph smith/irwin wunderman(rip/janis wignall'

In [99]:
additionelCollabs

ID                                                          108358
SearchName       [fuzzyics/(spd/m)], additional collaborators: ...
Year                                                          2021
Name             [FUZZYICS/(SPD/M)], Additional Collaborators: ...
Institution                                                  SPD/M
AuthorIndex                                                    3.0
Title            Siegel G…P FIRST “Google” Search-Engine and Si...
Division                                                      None
SessionIndex                                               72725.0
SessionType                                               attendee
LowerCaseName    [fuzzyics/(spd/m)], additional collaborators: ...
Name: 888, dtype: object

In [18]:
#df with error researchers, without the names with group
errorResearchers = errorResearchers[~errorResearchers.SearchName.isin(groupNames)]

In [42]:
#Search names containing university
uniNames = errorResearchers[errorResearchers.SearchName.str.contains('university')].SearchName.unique()

In [43]:
#df with error researchers, without the names with university in it
errorResearchers = errorResearchers[~errorResearchers.SearchName.isin(uniNames)]

In [47]:
#now look at 
#do as below, where we are looking at multiple. 
#look for department(check also if case matters. )

In [46]:
len(errorResearchers.SearchName.unique())

41945

In [ ]:
#remove team also

In [ ]:
researchers_notFound.LowerCaseName.unique()

In [ ]:
#researchers_notFound[researchers_notFound.SearchName.str.contains(',')]
researchers_notFound[researchers_notFound.SearchName.str.contains('department .* ,|university .* ,|department .* , .* university')]

In [ ]:
#check code from here
#try to run API again with researcherNotFound.pkl

#and try to figure out if there are any good reasons why some of them can't be found, such as having department and university in it, or group.
#we might need to simplify here and risk some errors

#check if there are any researchers from before API that is neither in researchers or researchersNotFound, 
#to make sure we did thing correct